In [14]:
import requests
from concurrent.futures import ThreadPoolExecutor
import time

In [6]:
rag_url = "http://127.0.0.1:8000/rag/{document_id}"

In [43]:
params = {
    "query": "What is the goal of this document?",
    "top_k": 10,
    "generation_engine": "huggingface",
}

In [44]:
def get_rag_response(document_id):
    return requests.get(rag_url.format(document_id=document_id), params=params, verify=False)

document_ids = ["CCLW.executive.8737.1424"] * 10
start = time.time()

with ThreadPoolExecutor(max_workers=10) as executor:
    responses = list(executor.map(get_rag_response, document_ids))

print(f"Time taken: {time.time() - start:.2f}s")
print(f"Responses: {responses}")
print(f"Status codes: {[resp.status_code for resp in responses]}")
print(f"Content lengths: {[len(resp.content) for resp in responses]}")
print(f"Content: {responses[0].json()}")


Time taken: 12.47s
Responses: [<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]
Status codes: [200, 200, 200, 200, 200, 200, 200, 200, 200, 200]
Content lengths: [117466, 116244, 117466, 117466, 117466, 117466, 117466, 116244, 116244, 117466]
Content: {'text': ' The goal of this document is to provide information on proposed mitigation actions and the role of monitoring, evaluation, and reporting in achieving emission reduction targets in various sectors, as outlined in the NCCAP 2018-2022 in Kenya.\n\n- The low target aligns with the proportional contribution required from each sector for the overall target to be achieved [1, 9].\n- The high target guides responsible ministries and agencies in planning and preparing for additional emission reductions [1, 2].\n- Attention should be paid to equal application of criteria, standards, and comparable use of form

In [45]:
texts = [r.json()["text"] for r in responses]
sources = ["\n".join(r.json()["sources_text"]) for r in responses]

assert len(set(sources)) == 1, "Sources are not the same"
assert len(set(texts)) == 1, "Texts are not the same"

AssertionError: Texts are not the same